Assume that we have the following influence diagram and we want to solve it using the jointree algorithm:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/model1.svg)

A quick review of the symbols in the model above:
- stochastic variables:
  - $r_i$: reward associated with item $i$
  - $w_i$: weight of item $i$
  - $h_i$: hidden variable affecting properties of item $i$
- decision variables:
  - $X_i$: binary decision variable indicating whether or not to take item $i$
- utility variables:
  - $U_i$: contribution of item $i$ to the objective function. These variables depend on their parents via this deterministic relation: $U_i = r_i * X_i$
  - $U_p$: a utility of $- \infty$ for assignments that violate the constraint, zero otherwise
- a deterministic variable representing the constraint $\sum_i w_i X_i < C$

Conversion of influence diagrams into jointrees is a bit different from that of Bayesian networks. However I only have the code for converting Bayesian networks to jointrees. So instead of the diagram above, I will compile the following network:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/bn1.svg)

In [2]:
push!(LOAD_PATH, pwd())
using JoinTreeInference.Node
using JoinTreeInference.Potential
using JoinTreeInference.parse_net
using JoinTreeInference.create_moral_graph
using JoinTreeInference.triangulate_graph
using JoinTreeInference.create_junction_tree

In [7]:
nodes, potentials = parse_net("./bn.net")
g_moral = create_moral_graph(nodes, potentials)

Dict{String,Set{String}} with 12 entries:
  "h1" => Set(String["r1","h2","w1"])
  "c"  => Set(String["x1","x2","w1","w2","up"])
  "w1" => Set(String["h1","x1","c","x2","w2"])
  "u1" => Set(String["x1","r1"])
  "up" => Set(String["c"])
  "h2" => Set(String["h1","r2","w2"])
  "x2" => Set(String["x1","c","r2","w1","w2","u2"])
  "w2" => Set(String["x1","c","h2","x2","w1"])
  "u2" => Set(String["r2","x2"])
  "x1" => Set(String["c","r1","x2","w1","w2","u1"])
  "r1" => Set(String["h1","x1","u1"])
  "r2" => Set(String["h2","x2","u2"])

In [13]:
g_triangulated, clusters = triangulate_graph(g_moral, nodes)

(Dict("h1"=>Set(String["x1","r1","h2","w1"]),"c"=>Set(String["x1","x2","w1","w2","up"]),"w1"=>Set(String["h1","x1","c","h2","x2","w2"]),"u1"=>Set(String["x1","r1"]),"up"=>Set(String["c"]),"h2"=>Set(String["h1","x1","r2","x2","w2","w1"]),"x2"=>Set(String["x1","c","r2","h2","w1","w2","u2"]),"w2"=>Set(String["x1","c","h2","x2","w1"]),"u2"=>Set(String["r2","x2"]),"x1"=>Set(String["h1","c","r1","h2","x2","w1","w2","u1"])…),Set{String}[Set(String["c","up"]),Set(String["x1","r1","u1"]),Set(String["r2","x2","u2"]),Set(String["x1","c","x2","w1","w2"]),Set(String["h1","x1","r1"]),Set(String["r2","h2","x2"]),Set(String["h1","x1","h2","w1"]),Set(String["x1","h2","x2","w2","w1"])])

In [15]:
g_triangulated

Dict{String,Set{String}} with 12 entries:
  "h1" => Set(String["x1","r1","h2","w1"])
  "c"  => Set(String["x1","x2","w1","w2","up"])
  "w1" => Set(String["h1","x1","c","h2","x2","w2"])
  "u1" => Set(String["x1","r1"])
  "up" => Set(String["c"])
  "h2" => Set(String["h1","x1","r2","x2","w2","w1"])
  "x2" => Set(String["x1","c","r2","h2","w1","w2","u2"])
  "w2" => Set(String["x1","c","h2","x2","w1"])
  "u2" => Set(String["r2","x2"])
  "x1" => Set(String["h1","c","r1","h2","x2","w1","w2","u1"])
  "r1" => Set(String["h1","x1","u1"])
  "r2" => Set(String["h2","x2","u2"])

In [14]:
clusters

8-element Array{Set{String},1}:
 Set(String["c","up"])                
 Set(String["x1","r1","u1"])          
 Set(String["r2","x2","u2"])          
 Set(String["x1","c","x2","w1","w2"]) 
 Set(String["h1","x1","r1"])          
 Set(String["r2","h2","x2"])          
 Set(String["h1","x1","h2","w1"])     
 Set(String["x1","h2","x2","w2","w1"])

In [11]:
create_junction_tree(clusters, nodes)

Dict{Int64,Set{Int64}} with 8 entries:
  7 => Set([8,5])
  4 => Set([8,1])
  2 => Set([5])
  3 => Set([6])
  5 => Set([7,2])
  8 => Set([7,4,6])
  6 => Set([3,8])
  1 => Set([4])